##### 问题:
我们想实现一个自定义的类，用来模仿普通的内建容器类型比如列表或者字典的行为。
但是，我们并不完全确定需要实现什么方法来完成。

##### 解决方案:
collections 库中定义了各种各样的抽象基类，当实现自定义的容器类时它们会非常有
用。为了说明清楚，假设我们希望自己的类能够支持迭代操作。要做到这点，只要简
单地从 collections.Iterable 中继承即可，就像下面这样：

In [61]:
import collections.abc
class A(collections.abc.Iterable):
   def __iter__(self):  #不定义下面报错
       pass


从 collections.Iterable 中继承的好处就是可以确保必须实现所有所需的特殊方法。如果
不这么做，那么在实例化时就会得到错误信息：

In [62]:
a = A()

<img src="2.png" width="140%">

要修正这个错误，只要在类中实现所需的__iter__()方法即可（参见 4.2 和 4.7 节）。

在 collections 库中还有其他一些值得一提的类，包括 Sequence、MutableSequence、
Mapping、MutableMapping、Set 以及 MutableSet。这些类中有许多是按照功能层次的
递增来进行排列的（例如，Container、Iterable、Sized、Sequence 以及 MutableSequence
就是一种递增式的排列）。再次说明，只要简单地对这些类进行实例化操作，就可以知
道需要实现哪些方法才能让自定义的容器具有相同的行为：

In [63]:
import collections.abc
# collections.abc.Sequence() 报错

<img src="1.png" width="140%">

下面有个简单的例子。我们在自定义类中实现了上述所需的方法，创建了一个 Sequence
类，且元素总是以排序后的顺序进行存储（我们的例子实现的不是很高效，但能说明
大意）：

In [64]:
import collections
import bisect

class SortedItems(collections.Sequence):
    def __init__(self,initial=None):
        self._items = sorted(initial) if initial is not None else []
    def __getitem__(self, item):
        return self._items[item]
    def __len__(self):
        return len(self._items)
    def add(self,value):
        bisect.insort(self._items,value)






下面是使用这个类的例子：

In [65]:
itemLt = SortedItems([5,1,2])
print(list(itemLt))         #[1, 2, 5]
print(itemLt[0])            # 1
itemLt.add(2)               #bisect功能非常强大,可以让序列保持有序,所以插入后依旧有序
print(list(itemLt))         #[1, 2, 2, 5]
print(len(itemLt))          #4

[1, 2, 5]
1
[1, 2, 2, 5]
4


可以看到，SortedItems 的实例所表现出的行为和一个普通的序列对象完全一样，并且
支持所有常见的操作，包括索引、迭代、len()、是否包含（in 操作符）甚至是分片。

顺便说一句，本节中用到的 bisect 模块能够方便地让列表中的元素保持有序。
bisect.insort()函数能够将元素插入到列表中且让列表仍然保持有序。

从 collections 库中提供的抽象基类继承，可确保我们的自定义容器实现了所有所需的方
法。但是，这种继承也便于我们做类型检查。

例如，我们的自定义容器将能够满足各种各样的类型检查：

In [66]:
items = SortedItems()
import collections.abc
print(isinstance(items, collections.abc.Iterable))
print(isinstance(items, collections.abc.Container)) 
print(isinstance(items, collections.abc.Mapping)) 

True
True
False


collections 模块中的许多抽象基类还针对常见的容器方法提供了默认实现。为了说明，
假设有一个类从 collections.MutableSequence 中继承而来，就像这样：

In [67]:

class Items(collections.MutableSequence):
    def __init__(self, initial=None):
        self._items = list(initial) if initial is not None else []

    def __getitem__(self, index):
        print('Getting:', index)
        return self._items[index]

    def __setitem__(self, index, value):
        print("setting [{}] to {}".format(index, value))
        self._items[index] = value

    def __delitem__(self, index):
        print("delete :", index)
        del self._items[index]

    def insert(self, index, value):
        print("Inserting:", index, value)
        self._items.insert(index, value)

    def __len__(self):
        print("Len")
        return len(self._items)


如果创建一个 Items 实例，就会发现它几乎支持列表所有的核心方法（例如 append()、
remove()、count()等）。这些方法在实现的时候只使用了所需要的那些特殊方法。下面
的交互式会话说明了这一点：

In [68]:
a = Items([1, 2, 3])
print(len(a))
a.append(4)     

Len
3
Len
Inserting: 3 4


本节仅仅只对 Python 的抽象类功能做了简要的介绍。numbers 模块中提供了与数值数
据类型相关的类似的抽象基类。要获得更多有关如何创建自己的抽象基类的信息，请
参阅 8.12 节。